In [1]:
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain
import encryption

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb
Importing Jupyter notebook from encryption.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO ETHEREUM GATEWAY

In [ ]:
web3 = blockchain.connect(settings)

### SERIALIZE THE SMART CONTRACT

In [7]:
auth_manager = blockchain.contract(latest['authmanager'], web3, settings)

### GENERATE & ASSIGN RSA ENCRYPTION KEY

In [ ]:
def generate_key(device):
    
    # GENERATE AN RSA ENCRYPTION KEYPAIR
    rsa_pubkey, rsa_privkey = encryption.generate_rsa_keys()
    
    # ASSIGN THE PUBLIC KEY TO THE DEVICE
    auth_manager.write({
        'func': 'assign_key',
        'params': [device, rsa_pubkey]
    })
    
    # SHA256 HASH THE PUBLIC KEY
    hashed_pubkey = utils.hash_data(rsa_pubkey)
    
    # STORE THE PRIVATE KEY AS A FILE
    utils.save_data(rsa_privkey, 'keys/' hashed_pubkey '.key')

### FETCH MOST RECENT BACKLOG

In [ ]:
def fetch_backlog():
    
    # FETCH RAW BACKLOG
    raw_backlog = auth_manager.read('fetch_backlog')
    
    # RETURN FILTERED BACKLOG
    return utils.filter_backlog(raw_backlog)

### MODIFICATION EVENT

In [ ]:
modification_event = auth_manager.event('modification')

### EVENT LOOP

In [ ]:
try:
    print('AWAITING REQUESTS...\n')
    
    # FETCH BACKLOG
    backlog = fetch_backlog()

    while(True):

        # VARIABLE MODIFICATION EVENT
        for event in modification_event.get_new_entries():
            
            # REFRESH BACKLOG
            backlog = fetch_backlog()

        # PERFORM TASKS IN BACKLOG
        for device in backlog:
            generate_key(device)
            
        # SLEEP FOR A SECOND
        time.sleep(1)

except:
    print('\nTHE PROCESS WAS MANUALLY KILLED')